In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
from google.colab import files
upload=files.upload()

Saving BOOK.csv to BOOK.csv


In [56]:
df=pd.read_csv('BOOK.csv',on_bad_lines='skip')
df

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [57]:
df.info

<bound method DataFrame.info of        bookID                                              title  \
0           1  Harry Potter and the Half-Blood Prince (Harry ...   
1           2  Harry Potter and the Order of the Phoenix (Har...   
2           4  Harry Potter and the Chamber of Secrets (Harry...   
3           5  Harry Potter and the Prisoner of Azkaban (Harr...   
4           8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
...       ...                                                ...   
11118   45631   Expelled from Eden: A William T. Vollmann Reader   
11119   45633                        You Bright and Risen Angels   
11120   45634                    The Ice-Shirt (Seven Dreams #1)   
11121   45639                                        Poor People   
11122   45641                        Las aventuras de Tom Sawyer   

                                                 authors  average_rating  \
0                             J.K. Rowling/Mary GrandPré            4.57   

In [58]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [59]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [60]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [61]:
df.isnull().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

Next, I had to decide on what criteria or which columns I thought would be helpful when determining suggested books that the user may like. I decided to use only 3 columns from the data set: title, authors, and publisher.

In [62]:
columns=['title','authors','publisher']

Once those important columns were identified, I created a function to concatenate the data from all three columns together and then used that function to store the data (combined features) into a new column.

In [63]:
def combine_features(data):
  features= []
  for i in range(0, data.shape[0]):
    features.append(data['title'][i] + ' ' +data['authors'][i]+ ' ' +data['publisher'][i])

  return features

In [64]:
df['combined_features']=combine_features(df)
df.head(3)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,combined_features
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Harry Potter and the Half-Blood Prince (Harry ...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Harry Potter and the Order of the Phoenix (Har...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,Harry Potter and the Chamber of Secrets (Harry...


Convert text from new column to a matrix of word counts.

In [65]:
cm=CountVectorizer().fit_transform(df['combined_features'])

In [66]:
#Comparing each book in the datset with every other book. So first row is comapred with the first column and see how similar it is.
# Here first book in row is similar to first book in first column.so it is has a 100 percent on 1 similarity score. 
cs=cosine_similarity(cm)
print(cs)

[[1.         0.8229512  0.73029674 ... 0.07856742 0.         0.        ]
 [0.8229512  1.         0.78881064 ... 0.14547859 0.         0.        ]
 [0.73029674 0.78881064 1.         ... 0.0860663  0.         0.        ]
 ...
 [0.07856742 0.14547859 0.0860663  ... 1.         0.2981424  0.        ]
 [0.         0.         0.         ... 0.2981424  1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [83]:
# Get the title of the book reader likes.
Title=df['title'][0]
Title

'Harry Potter and the Half-Blood Prince (Harry Potter  #6)'

In [84]:
book_id=df[df.title==Title]['bookID'].values[0]
book_id

1

In [85]:
# Create a list of tuples in the form(book_id,similarity scores)
scores=list(enumerate(cs[book_id]))
print(scores)

[(0, 0.8229511997978237), (1, 1.0000000000000002), (2, 0.7888106377466154), (3, 0.8743856497851876), (4, 0.6761234037828132), (5, 0.2727723627949905), (6, 0.6579516949597689), (7, 0.21821789023599242), (8, 0.23515854050088617), (9, 0.24743582965269678), (10, 0.23145502494313785), (11, 0.20025046972870353), (12, 0.07273929674533079), (13, 0.0), (14, 0.05292561240249632), (15, 0.0), (16, 0.0), (17, 0.12104550653376049), (18, 0.0), (19, 0.0), (20, 0.1690308509457033), (21, 0.3640126041546321), (22, 0.3857583749052298), (23, 0.4123930494211613), (24, 0.37424059428255985), (25, 0.3003757045930553), (26, 0.2817180849095055), (27, 0.0), (28, 0.0), (29, 0.10012523486435176), (30, 0.2727723627949905), (31, 0.1690308509457033), (32, 0.1315903389919538), (33, 0.0), (34, 0.0), (35, 0.17817416127494962), (36, 0.17817416127494962), (37, 0.19518001458970663), (38, 0.20701966780270625), (39, 0.17817416127494962), (40, 0.1543033499620919), (41, 0.17817416127494962), (42, 0.2182178902359924), (43, 0.145

In [92]:
# sort the list of similar books in descending order
sorted_scores=sorted(scores,key=lambda x:x[1] , reverse=True )
sorted_scores=sorted_scores[1:]
sorted_scores

[(3, 0.8743856497851876),
 (0, 0.8229511997978237),
 (4415, 0.8095238095238096),
 (2, 0.7888106377466154),
 (10675, 0.7324670207647143),
 (1233, 0.7092081432669753),
 (8873, 0.7008766440504623),
 (4, 0.6761234037828132),
 (6, 0.6579516949597689),
 (615, 0.6546536707079772),
 (10674, 0.6546536707079772),
 (988, 0.524890659167824),
 (4412, 0.50709255283711),
 (10343, 0.50709255283711),
 (10346, 0.50709255283711),
 (4562, 0.48304589153964794),
 (4411, 0.47633051162246687),
 (10345, 0.47633051162246687),
 (5350, 0.4605661864718383),
 (613, 0.45501575519329013),
 (7512, 0.43825049008927763),
 (10344, 0.4285714285714286),
 (72, 0.42766860172384985),
 (2879, 0.42433421239575275),
 (3858, 0.42365927286816174),
 (4263, 0.4199605255658081),
 (5239, 0.4199605255658081),
 (713, 0.4196434545311265),
 (8449, 0.4193139346887673),
 (10973, 0.41826814385706146),
 (6956, 0.41785544701867255),
 (8566, 0.41573970964154905),
 (4274, 0.4143631833949365),
 (9701, 0.4140393356054125),
 (23, 0.4123930494211613

In [94]:
# Create a loop to print the first 5 books from the sorted list
j=0
print('the 5 most recommended books to '+Title+' are:\n')
for item in sorted_scores:
  book_title=df[df.bookID==item[0]]['title'].values[0]
  print(j+1, book_title)
  j=j+1
  if j>=5:
    break

the 5 most recommended books to Harry Potter and the Half-Blood Prince (Harry Potter  #6) are:



IndexError: ignored